In [1]:
import torch

In [2]:
class GCN(torch.nn.Module):
    def __init__(self, sig_dimension,widths):
        super().__init__()
        self.f = sig_dimension
        self.widths = widths
        self.weights = []
        self.weights.append(torch.autograd.Variable(torch.rand(size = [self.f,widths[0]])-0.5,requires_grad = True))
        self.dropout = torch.nn.Dropout(0.02)
        
        for i in range(1,len(self.widths)):
            c = self.widths[i]
            self.weights.append(torch.autograd.Variable(torch.rand((self.widths[i-1],self.widths[i]))-0.5,requires_grad = True))

    def forward(self, adjM,node_sigM):
        curr_output = torch.tanh(torch.matmul(torch.matmul(adjM,node_sigM),self.weights[0])) #nxc
        #curr_output = self.dropout(curr_output)
        for i in range(1,len(self.widths)):
            curr_output = torch.tanh(torch.matmul(torch.matmul(adjM,curr_output),self.weights[i])) #nxc

        return curr_output

In [3]:
class MLP(torch.nn.Module):
    def __init__(self, input_shape,widths):
        super().__init__()
        self.input_shape = input_shape
        self.widths = widths
        self.weights = []
        self.weights.append(torch.autograd.Variable(torch.rand(size = [self.input_shape,widths[0]])-0.5,requires_grad = True))
        self.dropout = torch.nn.Dropout(0.02)
        
        for i in range(1,len(self.widths)):
            c = self.widths[i]
            self.weights.append(torch.autograd.Variable(torch.rand((self.widths[i-1],self.widths[i]))-0.5,requires_grad = True))

    def forward(self, x):
        x = torch.flatten(x,1,2)
        torch.matmul(x,self.weights[0])
        curr_output = torch.tanh(torch.matmul(x,self.weights[0])) #nxc
        #curr_output = self.dropout(curr_output)
        for i in range(1,len(self.widths)):
            curr_output = torch.tanh(torch.matmul(curr_output,self.weights[i])) #nxc

        return curr_output

In [1]:
class MLP_atk(torch.nn.Module):
    def __init__(self, input_shape, maxPoison,num_K, widths):
        super().__init__()
        self.input_shape = input_shape
        self.widths = widths
        self.maxPoison = maxPoison
        self.num_k = num_K
        
        
        self.weights = []
        self.weights.append(torch.autograd.Variable(torch.rand(size = [self.input_shape,widths[0]])-0.5,requires_grad = True))
        self.dropout = torch.nn.Dropout(0.02)
        
        for i in range(1,len(self.widths)):
            c = self.widths[i]
            self.weights.append(torch.autograd.Variable(torch.rand((self.widths[i-1],self.widths[i]))-0.5,requires_grad = True))

            
    def forward(self, x):
        x = torch.flatten(x,1,2)
        torch.matmul(x,self.weights[0])
        curr_output = torch.tanh(torch.matmul(x,self.weights[0])) #nxc
        #curr_output = self.dropout(curr_output)
        for i in range(1,len(self.widths)):
            curr_output = torch.tanh(torch.matmul(curr_output,self.weights[i])) #nxc
        
        magnitude = torch.reshape(torch.sigmoid(curr_output[:,1]),[curr_output.shape[0],1]) #the first output neuron encodes magnitude
        
        
        poisons_normed = torch.softmax(curr_output[:,1:],dim = 1) * self.maxPoison #normalize all poisons to within the maxPoison
        
        
        poisons = magnitude * poisons_normed
        
        poisons = torch.reshape(poisons,[curr_output.shape[0],K,3]) #multiply by the magnitude computed and reshape
     
            
        return poisons

SyntaxError: '(' was never closed (3136598398.py, line 27)